# DETOXIFICATION

In [1]:
%pip install -q torch
%pip install -q transformers[torch]
%pip install -q googletrans==3.1.0a0
%pip install -q fastparquet
%pip install sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.1.143 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.54.4 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 M

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Lectura de datos

Vamos a leer el dataset de entrenamiento. Este dataset esta formado por 400 frases tóxicas por idioma en 9 idiomas diferentes y sus versiones no tóxicas. Formaremos un dataset conjunto que contenga para cada frase, su idioma de origen, la frase tóxica y la frase de-toxificada.

In [3]:
splits = {'en': 'data/en-00000-of-00001.parquet', 'ru': 'data/ru-00000-of-00001.parquet', 'uk': 'data/uk-00000-of-00001.parquet', 'de': 'data/de-00000-of-00001.parquet', 'es': 'data/es-00000-of-00001.parquet', 'am': 'data/am-00000-of-00001.parquet', 'zh': 'data/zh-00000-of-00001.parquet', 'ar': 'data/ar-00000-of-00001.parquet', 'hi': 'data/hi-00000-of-00001.parquet'}
df_en = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["en"])
df_uk = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["uk"])
df_ru = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ru"])
df_de = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["de"])
df_es = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["es"])
df_am = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["am"])
df_zh = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["zh"])
df_ar = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ar"])
df_hi = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["hi"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
df_total = pd.concat([df_en, df_uk, df_ru, df_de, df_es, df_am, df_zh, df_ar, df_hi], keys=['en','uk', 'ru', 'de', 'es', 'am', 'zh-cn', 'ar', 'hi']).reset_index(level=[0])

In [5]:
df_total.sample(3)

,level_0,toxic_sentence,neutral_sentence
123,zh-cn,大风起兮云飞扬，安得猛士兮守四方，嘿畜，任何时候都要剿，不剿不行,大风起兮云飞扬，安得猛士兮守四方，黑人，个人认为任何时候都要受到一定惩罚，不惩罚不行
165,en,And why the fuck should i care?,And why should I care?
276,hi,तुम को संस्कृत से विरोध है तो कान में लूंड़ ठ...,तुमको संस्कृत से विरोध है तो अपने कान बंद कर लो |


### Train-test split

In [6]:
train, test = train_test_split(df_total, train_size=0.8, random_state=33)

X_train = list(train['toxic_sentence'])
y_train = list(train['neutral_sentence'])

X_test = list(test['toxic_sentence'])
y_test = list(test['neutral_sentence'])
len(X_test)

720

## Pipeline de detoxificación

Nuestro modelo de detoxificación tendrá un pipeline que consistirá de los siguientes pasos:

1. **Traducción**: Traduciremos las frases del idioma original al inglés
2. **Detoxificación**: El modelo detoxificará la frase en inglés
3. **Retro-traducción**: La frase detoxificada será traducida del inglés al idioma original

### Traducción y retrotraducción

Usaremos la libreria de google translate para crear un modelo que traduzca un texto de un lenguaje a otro, pasados por argumentos.

In [7]:
from googletrans import Translator

translator = Translator()

In [8]:
def translate(text, source_lang, dest_lang):
    if source_lang == dest_lang:
        return text

    sol = translator.translate(text , src = source_lang, dest=dest_lang).text
    return sol

In [9]:
test_trad = df_total.sample(5).to_numpy()

for sentence in test_trad:
    print(f"Original sentence ({sentence[0]}): {sentence[1]}")
    translation = translate(sentence[1], sentence[0], "en")
    print(f"Transalation: {translation}")
    print("-"*50)

Original sentence (en): why do you guys do this shit out west ?
Transalation: why do you guys do this shit out west ?
--------------------------------------------------
Original sentence (en): How could the capital of Israel not be part of Israel, you fucking brainwashed Arab dumbass.
Transalation: How could the capital of Israel not be part of Israel, you fucking brainwashed Arab dumbass.
--------------------------------------------------
Original sentence (en): Are u there dick, wars back on!!!!
Transalation: Are u there dick, wars back on!!!!
--------------------------------------------------
Original sentence (uk): бляха, вгамуйте хтось тимошенко. уже блювати від її коників хочеться.
Transalation: Goddammit, someone calm Tymoshenko down. I want to vomit from her grasshoppers.
--------------------------------------------------
Original sentence (uk): Інші просто долбайоби, бо я теж не шарю, за що ціна така.
Transalation: The others are just bullshit, because I also don't care why th

## Detoxificación

Ahora haremos un finetunning de un modelo de Huggingface para detoxificar el código.

In [10]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

num_training_layers = 2 # El número de capas que se entrenarán

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas n capas del decoder
for param in model.decoder.block[-num_training_layers:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ESTADÍSTICAS DEL ENTRENAMIENTO:")
print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

ESTADÍSTICAS DEL ENTRENAMIENTO:
Total de parámetros: 76,961,152
Parámetros entrenables: 22,744,064
Parámetros congelados: 54,217,088


#### Creación de los dataset

In [11]:
class DetoxificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        # Reemplazar pad_token_id con -100 para ignorar tokens de relleno en la pérdida
        item["labels"][item["labels"] == tokenizer.pad_token_id] = -100

        return item

In [12]:
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
y_train_tokenized = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

X_test_tokenized = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")
y_test_tokenized = tokenizer(y_test, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

print("Tamaño del dataset de entrenamiento:", len(train_dataset))
print("Ejemplo del dataset:", train_dataset[0])


# training_args = TrainingArguments(
#     output_dir='./checkpoints',
#     eval_strategy="no",
#     per_device_train_batch_size=128,
#     learning_rate=1e-4,
#     num_train_epochs=5,
#     report_to=["none"],
#     #fp16=True, # acelerar entrenaminento
# )


Tamaño del dataset de entrenamiento: 2880
Ejemplo del dataset: {'input_ids': tensor([ 3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,
         3,  2,  3,  2,  3,  2,  3,  2,  3,  2, 55,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 

## Entrenamiento

In [13]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm import tqdm

def train_t5(model, train_dataset, val_dataset, tokenizer, epochs=5, batch_size=8, lr=5e-5, device="cuda"):
    # Enviar modelo a dispositivo
    model.to(device)
    model.train()

    # DataLoaders para entrenamiento y validación
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizador y scheduler
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = epochs * len(train_loader)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    # Definir función de pérdida
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)  # Ignora los tokens -100

    # Loop de entrenamiento
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss = 0

        for batch in tqdm(train_loader, desc="Training"):
            # Preparar inputs y labels
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Adelante y cálculo de pérdida
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

            # Retropropagación y actualización de parámetros
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        print(f"Average training loss for epoch {epoch + 1}: {avg_loss}")

        # Validación después de cada época
        evaluate_t5(model, val_loader, loss_fn, device)
        model.save_pretrained("detox_model")

    return model

def evaluate_t5(model, val_loader, loss_fn, device):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Adelante y cálculo de pérdida
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation loss: {avg_val_loss}")
    model.train()


In [ ]:
trained_model = train_t5(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    tokenizer=tokenizer,
    epochs=10,
    batch_size=4,
    lr=1e-5,
    device=device
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10


Training:   3%|▎         | 25/720 [01:13<31:33,  2.72s/it]

In [ ]:
#guardar
trained_model.save_pretrained("detox_model")

In [ ]:
#cargar y probar
loaded_model = T5ForConditionalGeneration.from_pretrained("detox_model")
loaded_model.to(device)


frase="The only solution is to leave this shitty country."

loaded_model.eval()
input_ids = tokenizer(frase, return_tensors="pt").input_ids.to(device)
outputs = loaded_model.generate(input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)

